## 数组不相邻元素最大和

|0|1|2|3|4|5|6|
|-|-|-|-|-|-|
|1|2|4|1|7|8|3|

递归表达式：

OPT(i) = max{OPT(i-2)+arr\[i\], OPT(i-1)}

递归出口：

OPT(0) = arr\[0\]

OPT(1) = max{arr\[0\], arr\[1\]}

In [2]:
arr = [1, 2, 4, 1, 7, 8, 3]  # array

# recursive
# O(N^2)
def rec_opt(arr, i):
    if 0 == i:
        return arr[0]
    elif 1 == i:
        return max(arr[0], arr[1])
    else:
        A = rec_opt(arr, i - 2) + arr[i]
        B = rec_opt(arr, i - 1)
        return max(A, B)

print(rec_opt(arr, len(arr) - 1))

# iterative, dynamic programming
# O(N)
import numpy as np
def dp_opt(arr):
    opt = np.zeros(len(arr))
    opt[0] = arr[0]
    opt[1] = max(arr[0], arr[1])
    for i in range(2, len(arr)):
        A = opt[i - 2] + arr[i]
        B = opt[i - 1]
        opt[i] = max(A, B)
    return opt[-1]

print(dp_opt(arr))

15
15.0


## N-Sum问题

已知数组 arr = \[3, 34, 4, 12, 5, 2\]

能否找出其中和为 *S = 9* 的所有元素？
能就返回 *true*，否则返回 *false*

递归表达式：

Subset(i, s) = Subset(i - 1, s - arr\[i\]) **or** Subset(i - 1, s)

递归出口：

if s == 0: return True

if i == 0: return arr\[0\] == s

if arr\[i\] > s: return Subset(i - 1, s)

In [3]:
arr = [3, 34, 4, 12, 5, 2]

def rec_subset(arr, i, s):
    if 0 == s:
        return True
    elif 0 == i:
        return arr[0] == s
    elif arr[i] > s:
        return rec_subset(arr, i - 1, s)
    else:
        return rec_subset(arr, i - 1, s - arr[i]) or rec_subset(arr, i - 1, s)

print(rec_subset(arr, len(arr)-1, 9))
print(rec_subset(arr, len(arr)-1, 10))
print(rec_subset(arr, len(arr)-1, 11))
print(rec_subset(arr, len(arr)-1, 12))
print(rec_subset(arr, len(arr)-1, 13))

True
True
True
True
False


构造矩阵法：

|arr|i|0|1|2|3|4|5|6|7|8|9|
|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|
|3  |0|T|F|F|T|F|F|F|F|F|F|
|34 |1|T|...|
|4  |2|T||...|
|12 |3|T|||...|
|5  |4|T||||...|
|2  |5|T|||||...|

In [5]:
import numpy as np
def dp_subset(arr, S):
    subset = np.zeros((len(arr), S + 1), dtype=bool)
    subset[0, :] = False
    subset[:, 0] = True
    subset[0, arr[0]] = True
    for i in range(1, len(arr)):
        for s in range(1, S + 1):
            if arr[i] > s:
                subset[i, s] = subset[i - 1, s]
            else:
                subset[i, s] = subset[i - 1, s - arr[i]] or subset[i - 1, s]
    return subset[-1, -1]

print(dp_subset(arr, 9))
print(dp_subset(arr, 10))
print(dp_subset(arr, 11))
print(dp_subset(arr, 12))
print(dp_subset(arr, 13))

True
True
True
True
False


## 挖金矿

有一个国家发现了5座金矿，每座金矿的黄金储量不同，需要参与挖掘的工人数也不同。参与挖矿工人总数是10人。每座金矿要么全挖，要么不挖，不能派出一半人挖取一半金矿。要想得到尽可能多的黄金，应该选择挖取哪几座金矿？

{500金/5人，200金/3人，300金/4人，350金/3人，400金/5人}

**方法一：排列组合**

每一座金矿都有挖与不挖两种选择，如果有N座金矿，排列组合有2^N种选择。对所有可能性做遍历，排除那些使用工人数超过10的选择，在剩下的选择里找出获得金币数最多的选择。

时间复杂度为O(2^N)。

In [21]:
import itertools

g = [500, 200, 300, 350, 400]   # gold
w = [5, 3, 4, 3, 5]             # worker

index = list(range(0, len(g)))
gs = 0  # 黄金总数
for r in range(1, len(index)+1):
    # 选择不同数量时的组合数
    for ind in itertools.combinations(index, r):
        ws = sum([w[i] for i in ind])    # 工人总数
        if ws > 10:
            pass
        else:
            gs = max(gs, sum([g[i] for i in ind]))   # 挖取的黄金总数
print(gs)

900


动态规划

递归表达式：

OPT(i, N) = max{ OPT(i - 1, N - w\[i\]) + g\[i\], OPT(i - 1, N)}, N 为总人数

递归出口：

（1）只有一座金矿，只能挖这唯一的，而且人数不能超，得到的黄金就是这个矿的

if i == 0 & N >= w\[0\], OPT(0, N) = g\[0\]

（2）如果给定的工人数量不够挖取第1座金矿，也就是 *N < w\[0\]* 的时候，收获为0

if i == 0 & N < w\[0\], OPT(0, N) = 0

（3）可用工人数小于挖取该金矿需要人数

if i >= 1 & N < w\[i\], OPT(i, N) = OPT(i - 1, N)

In [25]:
g = [400, 500, 200, 300, 350]
w = [5, 5, 3, 4, 3]

def rec_opt(g, i, w, n):
    # i 是金矿数
    # n 是工人数
    if 0 == i and n < w[0]:
        return 0
    if 0 == i and n >= w[0]:
        return g[0]
    if i > 0 and n < w[i - 1]:
        return rec_opt(g, i - 1, w, n)
    
    a = rec_opt(g, i - 1, w, n)
    b = rec_opt(g, i - 1, w, n - w[i]) + g[i]
    return max(a, b)

print(rec_opt(g, len(g)-1, w, 10))

900


In [32]:
import numpy as np
# 动态规划，二维数组法

g = [400, 500, 200, 300, 350]
w = [5, 5, 3, 4, 3]

def dp_subset(g, ng, w, n):
    # ng is number of gold mines
    # n is number of workers
    col = n + 1
    preResults = np.zeros(col)    # 存放上一行结果
    results = np.zeros(col)       # 存放当前行结果

    # 填充边界
    for i in range(0, col):
        if i < w[0]:
            preResults[i] = 0
        else:
            preResults[i] = g[0]
    
    # 填充其余格子，从上一行推出下一行，外层循环金矿数量，内层工人数
    for i in range(0, ng):
        for j in range(0, col):
            if j < w[i]:
                results[j] = preResults[j]
            else:
                results[j] = max(preResults[j], preResults[j - w[i]] + g[i])
        
        for j in range(0, col):
            preResults[j] = results[j]
    
    return results[-1]

dp_subset(g, len(g), w, 10)

900.0